In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import BertPreTrainedModel, BertModel
from transformers import AutoConfig, AutoTokenizer

from sklearn import metrics
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer


In [ ]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import os
os.environ["WANDB_DISABLED"] = "true"
def softmax(x):
    """
    Compute softmax values for each set of scores in x.

    Parameters:
    x (array-like): Input array of scores.

    Returns:
    np.ndarray: Softmax probabilities.
    """
    exp_x = np.exp(x - np.max(x))  # Subtract max for numerical stability
    return exp_x / np.sum(exp_x)


In [ ]:
full_data_for_ranking = pd.read_excel('/content/drive/MyDrive/datasets/Budget_Speech_Sector_Company/sectorwise_budgetdaywise_performance_with_text_ranked_full.xlsx')

yr1 = 2020
yr2 = 2024

full_data_for_ranking['year_use'] = full_data_for_ranking['year'].apply(lambda x : int(str(x)[:4]))
full_data_for_ranking['text_segment'] = full_data_for_ranking['text_segment'].apply(lambda x : eval(x))
full_data_for_ranking['text_use'] = full_data_for_ranking.apply(lambda x : x.sector + ' [SEP] ' +' '.join(x.text_segment), axis = 1)

train = full_data_for_ranking[full_data_for_ranking['year_use']<yr1]
valid = full_data_for_ranking[(full_data_for_ranking['year_use']>=yr1) & (full_data_for_ranking['year_use']<yr2)]
test = full_data_for_ranking[full_data_for_ranking['year_use']>=yr2]
train.shape, valid.shape, test.shape

((318, 12), (41, 12), (70, 12))

In [ ]:
test.to_excel('/content/drive/MyDrive/datasets/Budget_Speech_Sector_Company/sectorwise_budgetdaywise_performance_with_text_ranked_full__TEST_ONLY.xlsx')

# BERT

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load and prepare dataset
#dataset = load_dataset('csv', data_files={'train': 'train.csv', 'test': 'test.csv'})

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=256
    )


train_dataset = Dataset.from_pandas(train[['text_use', 'mean_target_percentage_increment']].rename(columns = {'text_use' : 'text', 'mean_target_percentage_increment' : 'label'})).map(tokenize_function, batched=True)
eval_dataset = Dataset.from_pandas(valid[['text_use', 'mean_target_percentage_increment']].rename(columns = {'text_use' : 'text', 'mean_target_percentage_increment' : 'label'})).map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test[['text_use', 'mean_target_percentage_increment']].rename(columns = {'text_use' : 'text', 'mean_target_percentage_increment' : 'label'})).map(tokenize_function, batched=True)




In [ ]:
# Load BERT model for regression (num_labels=1 for continuous output)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
predictions = trainer.predict(eval_dataset)
preds = predictions.predictions.flatten()


In [ ]:
test_predictions = trainer.predict(test_dataset)

In [ ]:
from sklearn.metrics import ndcg_score

scores = np.array(test['mean_target_percentage_increment'])
test_scores_softmax= softmax(scores)

ndcg_score_all = []

dates = test['date'].unique()
test['predicted_probability'] = test_predictions.predictions.flatten()
test['mean_target_percentage_increment_transformed'] = test_scores_softmax

for date in dates:
  test_sub = test[test['date'] == date]
  true_relevance = np.asarray([test_sub['mean_target_percentage_increment_transformed'].to_list()])
  # we predict some scores (relevance) for the answers
  predicted_scores = np.asarray([test_sub['predicted_probability'].to_list()])
  ndcg_score_all.append(ndcg_score(true_relevance, predicted_scores))

<ipython-input-25-643d7d41622f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_probability'] = test_predictions.predictions.flatten()
<ipython-input-25-643d7d41622f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mean_target_percentage_increment_transformed'] = test_scores_softmax


In [ ]:
np.mean(ndcg_score_all)

0.9947527302376685

# RoBERTa

In [ ]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('FacebookAI/roberta-base')

def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=256
    )


train_dataset = Dataset.from_pandas(train[['text_use', 'mean_target_percentage_increment']].rename(columns = {'text_use' : 'text', 'mean_target_percentage_increment' : 'label'})).map(tokenize_function, batched=True)
eval_dataset = Dataset.from_pandas(valid[['text_use', 'mean_target_percentage_increment']].rename(columns = {'text_use' : 'text', 'mean_target_percentage_increment' : 'label'})).map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test[['text_use', 'mean_target_percentage_increment']].rename(columns = {'text_use' : 'text', 'mean_target_percentage_increment' : 'label'})).map(tokenize_function, batched=True)

# Load BERT model for regression (num_labels=1 for continuous output)
model = AutoModelForSequenceClassification.from_pretrained('FacebookAI/roberta-base', num_labels=1)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
predictions = trainer.predict(eval_dataset)
preds = predictions.predictions.flatten()

test_predictions = trainer.predict(test_dataset)


from sklearn.metrics import ndcg_score

scores = np.array(test['mean_target_percentage_increment'])
test_scores_softmax= softmax(scores)

ndcg_score_all = []

dates = test['date'].unique()
test['predicted_probability'] = test_predictions.predictions.flatten()
test['mean_target_percentage_increment_transformed'] = test_scores_softmax

for date in dates:
  test_sub = test[test['date'] == date]
  true_relevance = np.asarray([test_sub['mean_target_percentage_increment_transformed'].to_list()])
  # we predict some scores (relevance) for the answers
  predicted_scores = np.asarray([test_sub['predicted_probability'].to_list()])
  ndcg_score_all.append(ndcg_score(true_relevance, predicted_scores))

np.mean(ndcg_score_all)

Map:   0%|          | 0/318 [00:00<?, ? examples/s]

Map:   0%|          | 0/41 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


<ipython-input-30-fe1d42e83442>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_probability'] = test_predictions.predictions.flatten()
<ipython-input-30-fe1d42e83442>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mean_target_percentage_increment_transformed'] = test_scores_softmax


0.9951721514195451

# DeBERTa

In [ ]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=256
    )


train_dataset = Dataset.from_pandas(train[['text_use', 'mean_target_percentage_increment']].rename(columns = {'text_use' : 'text', 'mean_target_percentage_increment' : 'label'})).map(tokenize_function, batched=True)
eval_dataset = Dataset.from_pandas(valid[['text_use', 'mean_target_percentage_increment']].rename(columns = {'text_use' : 'text', 'mean_target_percentage_increment' : 'label'})).map(tokenize_function, batched=True)
test_dataset = Dataset.from_pandas(test[['text_use', 'mean_target_percentage_increment']].rename(columns = {'text_use' : 'text', 'mean_target_percentage_increment' : 'label'})).map(tokenize_function, batched=True)

# Load BERT model for regression (num_labels=1 for continuous output)
model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base', num_labels=1)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
predictions = trainer.predict(eval_dataset)
preds = predictions.predictions.flatten()

test_predictions = trainer.predict(test_dataset)


from sklearn.metrics import ndcg_score

scores = np.array(test['mean_target_percentage_increment'])
test_scores_softmax= softmax(scores)

ndcg_score_all = []

dates = test['date'].unique()
test['predicted_probability'] = test_predictions.predictions.flatten()
test['mean_target_percentage_increment_transformed'] = test_scores_softmax

for date in dates:
  test_sub = test[test['date'] == date]
  true_relevance = np.asarray([test_sub['mean_target_percentage_increment_transformed'].to_list()])
  # we predict some scores (relevance) for the answers
  predicted_scores = np.asarray([test_sub['predicted_probability'].to_list()])
  ndcg_score_all.append(ndcg_score(true_relevance, predicted_scores))

np.mean(ndcg_score_all)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/318 [00:00<?, ? examples/s]

...

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

<ipython-input-31-fddccb9c02b8>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_probability'] = test_predictions.predictions.flatten()
<ipython-input-31-fddccb9c02b8>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mean_target_percentage_increment_transformed'] = test_scores_softmax


0.9949204733688853